In [ ]:
#using the insurance.csv file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import LabelEncoder, OneHotEncoder



df= pd.read_csv(r"C:\\Users\meet_\OneDrive\Documents\vscode-projects\My-Individual-Project\jupyter_notebooks\Source Data\insurance.csv")
df = df.copy()
df.head()

df.isnull().sum()
df.shape
df.info()

df['smoker'] = df['smoker'].str.lower()

#Hypothesis 1 = Smoking has a direct impact on weight gain

""" 
Function to group BMI values in line with NHS defintions

"""

def bmi_group(bmi):
    if bmi < 18.5:
        return "underweight"
    elif 18.5 <= bmi < 24.9:
        return "normal weight"
    elif 25 <= bmi < 29.9:
        return "overweight"
    else:
        return "obese"
    
#add a new column in the df to group the bmi values
df["bmi_category"] = df["bmi"].apply(bmi_group)
df.head(25)

"""
Function to return a count of the bmi category for smokers
"""
def smoker_bmi_count(df): 
     smokers = df[df['smoker'] == "yes"]
     vc = smokers["bmi_category"].value_counts().reset_index()
     vc.columns = ["bmi_category","count_smoker"]
     return vc

"""
Function to return a count of the bmi category for nonsmokers
"""
def nonsmoker_bmi_count(df): 
     non_smokers = df[df['smoker'] == "no"]
     vc = non_smokers["bmi_category"].value_counts().reset_index()
     vc.columns = ["bmi_category","count_nonsmoker"]
     return vc    



smoker_bmi_df = smoker_bmi_count(df)
nonsmoker_bmi_df = nonsmoker_bmi_count(df)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
